In [ ]:
using_colab = True   # Set True if running in Google Colab. Set to False if running in Jupyter Lab

# Forecasting Net Prophet

You’re a growth analyst at [MercadoLibre](http://investor.mercadolibre.com/investor-relations). With over 200 million users, MercadoLibre is the most popular e-commerce site in Latin America. You've been tasked with analyzing the company's financial and user data in clever ways to make the company grow. So, you want to find out if the ability to predict search traffic can translate into the ability to successfully trade the stock.

Instructions

This section divides the instructions for this Challenge into four steps and an optional fifth step, as follows:

* Step 1: Find unusual patterns in hourly Google search traffic

* Step 2: Mine the search traffic data for seasonality

* Step 3: Relate the search traffic to stock price patterns

* Step 4: Create a time series model with Prophet

* Step 5 (optional): Forecast revenue by using time series models

The following subsections detail these steps.

## Step 1: Find Unusual Patterns in Hourly Google Search Traffic

The data science manager asks if the Google search traffic for the company links to any financial events at the company. Or, does the search traffic data just present random noise? To answer this question, pick out any unusual patterns in the Google search data for the company, and connect them to the corporate financial events.

To do so, complete the following steps:

1. Read the search data into a DataFrame, and then slice the data to just the month of May 2020. (During this month, MercadoLibre released its quarterly financial results.) Use hvPlot to visualize the results. Do any unusual patterns exist?

2. Calculate the total search traffic for the month, and then compare the value to the monthly median across all months. Did the Google search traffic increase during the month that MercadoLibre released its financial results?

## Step 2: Mine the Search Traffic Data for Seasonality

Marketing realizes that they can use the hourly search data, too. If they can track and predict interest in the company and its platform for any time of day, they can focus their marketing efforts around the times that have the most traffic. This will get a greater return on investment (ROI) from their marketing budget.

To that end, you want to mine the search traffic data for predictable seasonal patterns of interest in the company. To do so, complete the following steps:

1. Group the hourly search data to plot the average traffic by the day of the week (for example, Monday vs. Friday).

2. Using hvPlot, visualize this traffic as a heatmap, referencing the `index.hour` as the x-axis and the `index.dayofweek` as the y-axis. Does any day-of-week effect that you observe concentrate in just a few hours of that day?

3. Group the search data by the week of the year. Does the search traffic tend to increase during the winter holiday period (weeks 40 through 52)?

## Step 3: Relate the Search Traffic to Stock Price Patterns

You mention your work on the search traffic data during a meeting with people in the finance group at the company. They want to know if any relationship between the search data and the company stock price exists, and they ask if you can investigate.

To do so, complete the following steps:

1. Read in and plot the stock price data. Concatenate the stock price data to the search data in a single DataFrame.

2. Market events emerged during the year of 2020 that many companies found difficult. But, after the initial shock to global financial markets, new customers and revenue increased for e-commerce platforms. Slice the data to just the first half of 2020 (`2020-01` to `2020-06` in the DataFrame), and then use hvPlot to plot the data. Do both time series indicate a common trend that’s consistent with this narrative?

3. Create a new column in the DataFrame named “Lagged Search Trends” that offsets, or shifts, the search traffic by one hour. Create two additional columns:

    * “Stock Volatility”, which holds an exponentially weighted four-hour rolling average of the company’s stock volatility

    * “Hourly Stock Return”, which holds the percent change of the company's stock price on an hourly basis

4. Review the time series correlation, and then answer the following question: Does a predictable relationship exist between the lagged search traffic and the stock volatility or between the lagged search traffic and the stock price returns?

## Step 4: Create a Time Series Model with Prophet

Now, you need to produce a time series model that analyzes and forecasts patterns in the hourly search data. To do so, complete the following steps:

1. Set up the Google search data for a Prophet forecasting model.

2. After estimating the model, plot the forecast. How's the near-term forecast for the popularity of MercadoLibre?

3. Plot the individual time series components of the model to answer the following questions:

    * What time of day exhibits the greatest popularity?

    * Which day of the week gets the most search traffic?

    * What's the lowest point for search traffic in the calendar year?

## Step 5 (Optional): Forecast Revenue by Using Time Series Models

A few weeks after your initial analysis, the finance group follows up to find out if you can help them solve a different problem. Your fame as a growth analyst in the company continues to grow!

Specifically, the finance group wants a forecast of the total sales for the next quarter. This will dramatically increase their ability to plan budgets and to help guide expectations for the company investors.

To do so, complete the following steps:

1. Read in the daily historical sales (that is, revenue) figures, and then apply a Prophet model to the data.

2. Interpret the model output to identify any seasonal patterns in the company's revenue. For example, what are the peak revenue days? (Mondays? Fridays? Something else?)

3. Produce a sales forecast for the finance group. Give them a number for the expected total sales in the next quarter. Include the best- and worst-case scenarios to help them make better plans.


## Install and import the required libraries and dependencies

## Step 1: Find Unusual Patterns in Hourly Google Search Traffic

The data science manager asks if the Google search traffic for the company links to any financial events at the company. Or, does the search traffic data just present random noise? To answer this question, pick out any unusual patterns in the Google search data for the company, and connect them to the corporate financial events.

To do so, complete the following steps:

1. Read the search data into a DataFrame, and then slice the data to just the month of May 2020. (During this month, MercadoLibre released its quarterly financial results.) Use hvPlot to visualize the results. Do any unusual patterns exist?

2. Calculate the total search traffic for the month, and then compare the value to the monthly median across all months. Did the Google search traffic increase during the month that MercadoLibre released its financial results?


In [ ]:
# Install the required libraries
!pip install prophet
!pip install hvplot
!pip install holoviews

# pscript needed for custom hvplot axis formatter
!pip install pscript
!pip install jupyter_bokeh

In [ ]:
# Import the required libraries and dependencies
if not using_colab:
    from pathlib import Path
    
import numpy as np
import pandas as pd
import holoviews as hv
from prophet import Prophet
import hvplot.pandas
import datetime as dt
from bokeh.models import DatetimeTickFormatter, NumeralTickFormatter # Used for formatting the chart axes

%matplotlib inline

#### Step 1: Read the search data into a DataFrame, and then slice the data to just the month of May 2020. (During this month, MercadoLibre released its quarterly financial results.) Use hvPlot to visualize the results. Do any unusual patterns exist?

In [ ]:
# Upload the "google_hourly_search_trends.csv" file into Colab, then store in a Pandas DataFrame
# Set the "Date" column as the Datetime Index.

mercado_trends_path_filename = "google_hourly_search_trends.csv"
if using_colab:
    from google.colab import files
    uploaded = files.upload()

    df_mercado_trends = pd.read_csv(mercado_trends_path_filename) # Read the specified file into the dataframe
else:
    mercado_trends_path =  Path("./Resources/" + mercado_trends_path_filename)
    df_mercado_trends =  pd.read_csv(mercado_trends_path) # Read the specified file into the dataframe

# Review the first and last five rows of the DataFrame
display("Head",df_mercado_trends.head()) # Show the first 5 rows
display("Tail",df_mercado_trends.tail()) # Show the last 5 rows

In [ ]:
# Convert Date strings into datetime objects and set the datetime as the index
df_mercado_trends = df_mercado_trends.set_index(pd.to_datetime(df_mercado_trends["Date"], infer_datetime_format=True))

df_mercado_trends = df_mercado_trends.drop(columns=["Date"]) # Drop the original Date column.... ("columns=labels" is equivalent to "labels, axis=1").
 
# Review the first and last five rows of the DataFrame
display("Head",df_mercado_trends.head()) # Show the first 5 rows
display("Tail",df_mercado_trends.tail()) # Show the last 5 rows

In [ ]:
# Review the data types of the DataFrame using the info function
df_mercado_trends.info() # Show information about the dataframe


In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Slice the DataFrame to just the month of May 2020
df_may_2020 = df_mercado_trends.loc["2020-05"]  # Assign the slice of May 2020


# Use hvPlot to visualize the data for May 2020
# Implementation notes: decided to group by day, as plotting just on the raw data was not showing any significant trends during the month.
fig1_label = "Figure 1 - Daily Google search trends for MercadoLibre - May 2020"  # Set the title
fig1 = df_may_2020.groupby(df_may_2020.index.date).sum().hvplot( # Plot the search trends with a descriptive label
    label=fig1_label,                                            # Set the title
    xlabel="Date",                                               # Label the x axis
    ylabel="Search Trends",                                      # Label the y axis
    grid=True,                                                   # Show the grid
    ).opts(                                                      # Format the x and y axes
        xformatter=DatetimeTickFormatter(days="%d/%m/%Y"),       # Show date in dd/mm/yyyy format
        yformatter=NumeralTickFormatter(format="0,0"),           # Add thousands separator to the y axis
        )
fig1
# xformatter & yformatter code snippets from https://docs.bokeh.org/en/latest/docs/reference/models/formatters.html#bokeh.models.DatetimeTickFormatter

#### Step 2: Calculate the total search traffic for the month, and then compare the value to the monthly median across all months. Did the Google search traffic increase during the month that MercadoLibre released its financial results?

In [ ]:
# Calculate the sum of the total search traffic for May 2020
traffic_may_2020 = df_may_2020["Search Trends"].sum() # Sum the total traffic for May 2020

# View the traffic_may_2020 value
print(f"Total search traffic for May 2020 was {traffic_may_2020:0,.0f}") # Display the total traffic for May 2020 with formatting

In [ ]:
# Calcluate the monthly median search traffic across all months
# Group the DataFrame by index year and then index month, chain the sum and then the median functions
median_monthly_traffic = df_mercado_trends["Search Trends"].groupby([df_mercado_trends.index.year, df_mercado_trends.index.month ]).sum().median() # Find the median across all months

# View the median_monthly_traffic value
print(f"Median search traffic across all months was {median_monthly_traffic:0,.0f}") # Display the median traffic with formatting

In [ ]:
# Compare the seach traffic for the month of May 2020 to the overall monthly median value
may2020_vs_all_months_delta = traffic_may_2020 - median_monthly_traffic                           # Work out the delta
may2020_vs_all_months_pct = round(may2020_vs_all_months_delta / median_monthly_traffic * 100, 2)  # Work out the delta %

# Present the findings
print(f"There was a change of {may2020_vs_all_months_pct:-0,.2f}% (Δ={may2020_vs_all_months_delta:-0,.0f}) in volume during May 2020 (∑={traffic_may_2020:0,.0f}) when compared to the median of all months (∑={median_monthly_traffic:0,.0f})")

##### Answer the following question:

**Question:** Did the Google search traffic increase during the month that MercadoLibre released its financial results?

**Answer:** Yes, there appears to have been an increase of 8.55% (Δ=3,008) in volume during May 2020 (∑=38,181) when compared to the median of all months (∑=35,172)

## Step 2: Mine the Search Traffic Data for Seasonality

Marketing realizes that they can use the hourly search data, too. If they can track and predict interest in the company and its platform for any time of day, they can focus their marketing efforts around the times that have the most traffic. This will get a greater return on investment (ROI) from their marketing budget.

To that end, you want to mine the search traffic data for predictable seasonal patterns of interest in the company. To do so, complete the following steps:

1. Group the hourly search data to plot the average traffic by the day of the week (for example, Monday vs. Friday).

2. Using hvPlot, visualize this traffic as a heatmap, referencing the `index.hour` as the x-axis and the `index.dayofweek` as the y-axis. Does any day-of-week effect that you observe concentrate in just a few hours of that day?

3. Group the search data by the week of the year. Does the search traffic tend to increase during the winter holiday period (weeks 40 through 52)?


#### Step 1: Group the hourly search data to plot the average traffic by the day of the week (for example, Monday vs. Friday).

In [ ]:
# Create tuples to substitute plot axis labels. Code reference as per https://hvplot.holoviz.org/user_guide/Customization.html

# Create a list of tuples to replace the day of week number with the day of week name.
ticks_day_of_week_labels = [(0,"Monday"), (1,"Tuesday"), (2,"Wednesday"), (3,"Thursday"), (4,"Friday"), (5,"Saturday"), (6,"Sunday")]

# Create a list of tuples to replace the hour labels
ticks_time_of_day_labels = [(0,"Midnight"), (4,"4am"), (8,"8am"), (12,"Midday"), (16,"4pm"), (20,"8pm"), (23,"11pm")]

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Group the hourly search data to plot (use hvPlot) the average traffic by the day of week
grouping = df_mercado_trends.groupby(by=[df_mercado_trends.index.dayofweek])["Search Trends"].mean()    # Define a grouping level based on week day (e.g., Mon = 0, Tues = 1 etc)

display("Day of Week Trend Data")
display(grouping) # show the grouping results

fig2_label = "Figure 2 - Average search traffic by day of week" # Set the content for the plot's title

fig2 = grouping.hvplot(    # Plot the search trends by day of week
    x="index.day_name",                                       # Set the x axis data source
    y="Search Trends",                               # Set the y axis data source
    label=fig2_label,                                # Set the title
    xlabel="Day of Week",                            # Label the x axis
    ylabel="Search Trends (avg)",                    # Label the y axis
    grid=True,                                       # Show the grid
    xticks = ticks_day_of_week_labels,               # Remap the numeric dayofweek labels to the day name of the week
    yformatter=NumeralTickFormatter(format="0,0")    # Add thousands separator to the y axis
)
display(fig2) # Show the plot

#### Step 2: Using hvPlot, visualize this traffic as a heatmap, referencing the `index.hour` as the x-axis and the `index.dayofweek` as the y-axis. Does any day-of-week effect that you observe concentrate in just a few hours of that day?

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Use hvPlot to visualize the hour of the day and day of week search traffic as a heatmap.
fig3_label = "Figure 3 - Search traffic heat map - hour of day by day of week"  # Set the content for the plot's title
fig3 = df_mercado_trends.hvplot.heatmap(label=fig3_label,                       # Set the label
                                        x="index.hour",                         # x axis data source (as instructed)
                                        y="index.dayofweek",                    # y axis data source (as instructed)
                                        C="Search Trends",                      # data for heat map
                                        cmap="reds",                            # use the red colour scheme for good contrast
                                        xlabel="Hour of Day",                   # Label the x axis
                                        ylabel="Day of Week",                   # Label the y axis
                                        yticks=ticks_day_of_week_labels,        # Remap the dayofweek number (eg 0, 1) labels to the day name of the week (eg Monday, Tuesday)
                                        xticks=ticks_time_of_day_labels,        # Remap the hour number (eg 0, 1..23) to a friendlier format
                                        ).aggregate(function=np.mean)

In [ ]:
hv.extension('bokeh')
display(fig3) # Display figure 3, the heatmap

##### Answer the following question:

**Question:** Does any day-of-week effect that you observe concentrate in just a few hours of that day?

**Answer:** As can been seen in Figure 3, there is hardly any activity between 6am and 11am. There is a gradual increase in activity from midday onwards to late evening, at which point the highest concentration of activity can be observed between 10pm and 2am. Activity declines fairly rapidly from 3am onwards.

#### Step 3: Group the search data by the week of the year. Does the search traffic tend to increase during the winter holiday period (weeks 40 through 52)?

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Group the hourly search data to plot (use hvPlot) the average traffic by the week of the year
fig4_grouping = [ pd.Index(df_mercado_trends.index.isocalendar().week) ]     # Group by week of the year using pd.Int64Index(idx.isocalendar().week) to overcome # FutureWarning: weekofyear and week have have been deprecated warning

fig4_label = "Figure 4 - Total search traffic by week of year" # Set the content for the plot's title
fig4 = df_mercado_trends.groupby( by = fig4_grouping ).sum().hvplot(      # Plot the search trends by day of week
     label=fig4_label,                                # Set the title
     xlabel="Week of Year",                           # Label the x axis
     ylabel="Search Trends (total)",                  # Label the y axis
     grid=True,                                       # Show the grid
    yformatter=NumeralTickFormatter(format="0,0")     # Add thousands separator to the y axis
)

display(fig4) # show the plot

##### Answer the following question:

**Question:** Does the search traffic tend to increase during the winter holiday period (weeks 40 through 52)?

**Answer:** Mercadolibre's search traffic is at its highest between weeks 21 and weeks 36, coinciding with the northern hemisphere's summer. From week 38 there is a significant drop in total volume. Therefore the search traffic tends to *DECREASE* during the (northern hemisphere's) winter holiday period (weeks 40 through 52).

## Step 3: Relate the Search Traffic to Stock Price Patterns

You mention your work on the search traffic data during a meeting with people in the finance group at the company. They want to know if any relationship between the search data and the company stock price exists, and they ask if you can investigate.

To do so, complete the following steps:

1. Read in and plot the stock price data. Concatenate the stock price data to the search data in a single DataFrame.

2. Market events emerged during the year of 2020 that many companies found difficult. But, after the initial shock to global financial markets, new customers and revenue increased for e-commerce platforms. Slice the data to just the first half of 2020 (`2020-01` to `2020-06` in the DataFrame), and then use hvPlot to plot the data. Do both time series indicate a common trend that’s consistent with this narrative?

3. Create a new column in the DataFrame named “Lagged Search Trends” that offsets, or shifts, the search traffic by one hour. Create two additional columns:

    * “Stock Volatility”, which holds an exponentially weighted four-hour rolling average of the company’s stock volatility

    * “Hourly Stock Return”, which holds the percent change of the company's stock price on an hourly basis

4. Review the time series correlation, and then answer the following question: Does a predictable relationship exist between the lagged search traffic and the stock volatility or between the lagged search traffic and the stock price returns?


#### Step 1: Read in and plot the stock price data. Concatenate the stock price data to the search data in a single DataFrame.

In [ ]:
# Upload the "mercado_stock_price.csv" file into Colab, then store in a Pandas DataFrame
# Set the "date" column as the Datetime Index.

mercado_stock_path_filename = "mercado_stock_price.csv"    # set the filename to load
if using_colab:                                            # If in the Colab environment...
    from google.colab import files                         #     import colab library
    uploaded = files.upload()                              #     upload the file via colab

    df_mercado_stock =  pd.read_csv(mercado_stock_path_filename) # Read the specified file into the dataframe
else:
    mercado_stock_path = Path("./Resources/" + mercado_stock_path_filename )     # Get the path in the OS's format
    df_mercado_stock = pd.read_csv(mercado_stock_path) # Read the specified file into the dataframe

df_mercado_stock = df_mercado_stock.set_index(pd.to_datetime(df_mercado_stock["date"], infer_datetime_format=True) )
df_mercado_stock = df_mercado_stock.drop(columns=["date"]) # Drop the original Date column.... ("columns=labels" is equivalent to "labels, axis=1").

# Review the first and last five rows of the DataFrame
display("Head", df_mercado_stock.head()) # Show the first 5 rows
display("Tail", df_mercado_stock.tail()) # Show the last 5 rows

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Use hvPlot to visualize the closing price of the df_mercado_stock DataFrame
fig5_label = "Figure 5 - Mercadolibre, Inc. (MELI) Closing Stock Price" # Set the content for the plot's title

fig5 = df_mercado_stock.hvplot(                      # Plot the graph
    label=fig5_label,                                # Set the title
    xlabel="Date",                                   # Label the x axis
    ylabel="Closing Price (USD)",                    # Label the y axis
    grid=True,                                       # Show the grid
    yformatter=NumeralTickFormatter(format="0,0")    # Add thousands separator to the y axis
)
display(fig5) # Show the plot

In [ ]:
# Concatenate the df_mercado_stock DataFrame with the df_mercado_trends DataFrame
# Concatenate the DataFrame by columns (axis=1), and drop and rows with only one column of data
mercado_stock_trends_df = pd.concat([df_mercado_stock, df_mercado_trends], axis = 1).dropna()  # Join the two dataframes by column, then drop the redundant date index


# View the first and last five rows of the DataFrame
display("Head", mercado_stock_trends_df.head())
display("Tail", mercado_stock_trends_df.tail())

#### Step 2: Market events emerged during the year of 2020 that many companies found difficult. But, after the initial shock to global financial markets, new customers and revenue increased for e-commerce platforms. Slice the data to just the first half of 2020 (`2020-01` to `2020-06` in the DataFrame), and then use hvPlot to plot the data. Do both time series indicate a common trend that’s consistent with this narrative?

In [ ]:
# For the combined dataframe, slice to just the first half of 2020 (2020-01 through 2020-06)
first_half_2020 = mercado_stock_trends_df.loc['2020-01' : '2020-06'] # Create the slice from the desired time period

# View the first and last five rows of first_half_2020 DataFrame
display("Head", first_half_2020.head())
display("Tail", first_half_2020.tail())

In [ ]:
# Since the stock data has hourly closing prices, aggregate the stock and trend information by day to have a clearer view.
first_half_2020 = first_half_2020.groupby(first_half_2020.index.date).agg({'close':'mean', 'Search Trends':'sum'}).copy() # Average the Close and sum the Search trends by day

# View the first and last five rows of first_half_2020 DataFrame
display("Head", first_half_2020.head())
display("Tail", first_half_2020.tail())

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Use hvPlot to visualize the close and Search Trends data
# Plot each column on a separate axes using the following syntax
# `hvplot(shared_axes=False, subplots=True).cols(1)`

# Implementation notes. Have used separate plots to start with so that the title and Y axis labels can be specified for a more informative plot
# Set separate titles for each plot's
fig6_stock_label = "Figure 6a - Mercadolibre Closing Stock Price" 
fig6_trend_label = "Figure 6b - Mercadolibre Search Trends"

# Create the Stock plot
fig6a = first_half_2020.hvplot(                      # Plot the graph
    x="index",                                       # Use the index's date  as the X axis
    y="close",                                       # Use the average daily closing price column as the Y axis
    shared_axes=False,                               # Dont share the axes
    subplots=True,                                   # Allow subplots
    label=fig6_stock_label,                          # Set the title    
    xlabel="Date",                                   # Label the x axis
    ylabel="Closing Price (USD)",                    # Label the y axis
    grid=True,                                       # Show the grid
    yformatter=NumeralTickFormatter(format="0,0")    # Add thousands separator to the y axis
)

# Create the Search Trend plot
fig6b = first_half_2020.hvplot(                      # Plot the graph
    x="index",                                       # Use the index's date  as the X axis
    y="Search Trends",                               # Use the Search Trends column as the Y axis
    shared_axes=False,                               # Dont share the axes
    subplots=True,                                   # Allow subplots
    label=fig6_trend_label,                          # Set the title    
    xlabel="Date",                                   # Label the x axis
    ylabel="Daily Search Trends",                    # Label the y axis
    grid=True,                                       # Show the grid
    yformatter=NumeralTickFormatter(format="0,0")    # Add thousands separator to the y axis
)

display( (fig6a+fig6b).cols(1) ) # Display the 2 plots in 1 bokeh panel on separate rows

##### Answer the following question:

**Question:** Do both time series indicate a common trend that’s consistent with this narrative?

**Answer:** The stock price dropped steadily by an eventual 50% within the period from late February to early April 2020, during which time search trends initially bucked the trend. Then came a sudden drop in search trends which fluctuated at close to low points for another month, during which time the share price gradually increased. A dramatic spike in searches in May 2020 coincided with an almost 20% jump in share price. After the dramatic spike search trends dropped by 25% approximately however the share price continued to increase. The trend appears to be consistent with the narrative in that business returned after the initial market shock.

#### Step 3: Create a new column in the DataFrame named “Lagged Search Trends” that offsets, or shifts, the search traffic by one hour. Create two additional columns:

* “Stock Volatility”, which holds an exponentially weighted four-hour rolling average of the company’s stock volatility

* “Hourly Stock Return”, which holds the percent change of the company's stock price on an hourly basis


In [ ]:
# Create a new column in the mercado_stock_trends_df DataFrame called Lagged Search Trends
# This column should shift the Search Trends information by one hour
mercado_stock_trends_df["Lagged Search Trends"] = mercado_stock_trends_df["Search Trends"].shift(1)  # Add the new Lagged column based on the original Search Trends shifted down by 1 row
mercado_stock_trends_df

In [ ]:
# Create a new column in the mercado_stock_trends_df DataFrame called Stock Volatility
# This column should calculate the standard deviation of the closing stock price return data over a 4 period rolling window
mercado_stock_trends_df["Stock Volatility"] = mercado_stock_trends_df["close"].rolling( window=4 ).std() # Add the new Volatility column based on std over a 4 hr period
mercado_stock_trends_df.head() # show the data frame

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Use hvPlot to visualize the stock volatility
fig7_label = "Figure 7 - Mercadolibre Stock Price Rolling 4 hr Volatility"  # Set the plot's title

# Create the Stock plot
fig7 = mercado_stock_trends_df["Stock Volatility"].hvplot(    # Plot the graph
    label=fig7_label,                                         # Set the title    
    xlabel="Date",                                            # Label the x axis
    ylabel="Volatility (std)",                                # Label the y axis
    grid=True,                                                # Show the grid
    yformatter=NumeralTickFormatter(format="0,0")             # Add thousands separator to the y axis
)

display(fig7) # Show the plot

**Solution Note:** Note how volatility spiked, and tended to stay high, during the first half of 2020. This is a common characteristic of volatility in stock returns worldwide: high volatility days tend to be followed by yet more high volatility days. When it rains, it pours.

In [ ]:
# Create a new column in the mercado_stock_trends_df DataFrame called Hourly Stock Return
# This column should calculate hourly return percentage of the closing price
mercado_stock_trends_df["Hourly Stock Return"] = mercado_stock_trends_df["close"].pct_change() # Create a new Hourly Stock Return column based on % change

In [ ]:
# View the first and last five rows of the mercado_stock_trends_df DataFrame
display("Head", mercado_stock_trends_df.head())
display("Tail", mercado_stock_trends_df.tail())

#### Step 4: Review the time series correlation, and then answer the following question: Does a predictable relationship exist between the lagged search traffic and the stock volatility or between the lagged search traffic and the stock price returns?

In [ ]:
# Construct correlation table of Stock Volatility, Lagged Search Trends, and Hourly Stock Return

display("Table 1 - Correlation table of Mercadolibre's Stock Volatility, Lagged Search Trends, and Hourly Stock Return" ) # Show a title for the table
display(mercado_stock_trends_df[["Stock Volatility", "Lagged Search Trends", "Hourly Stock Return"]].corr()) # Create a correlation table between the values. Note .corr ignores NA/nulls

##### Answer the following question:


**Question:** Does a predictable relationship exist between the lagged search traffic and the stock volatility or between the lagged search traffic and the stock price returns?

**Answer:** Table 1 presents a weak negative correlation between the lagged search traffic and the stock volatility. Whereas, there is a weak positive correlation between lagged search traffic and the stock price returns.

## Step 4: Create a Time Series Model with Prophet

Now, you need to produce a time series model that analyzes and forecasts patterns in the hourly search data. To do so, complete the following steps:

1. Set up the Google search data for a Prophet forecasting model.

2. After estimating the model, plot the forecast. How's the near-term forecast for the popularity of MercadoLibre?

3. Plot the individual time series components of the model to answer the following questions:

    * What time of day exhibits the greatest popularity?

    * Which day of the week gets the most search traffic?

    * What's the lowest point for search traffic in the calendar year?


#### Step 1: Set up the Google search data for a Prophet forecasting model.

In [ ]:
# Using the df_mercado_trends DataFrame, reset the index so the date information is no longer the index
mercado_prophet_df = df_mercado_trends.reset_index() # Create a new dataframe for use with Prophet, which needs the index reset

# Label the columns ds and y so that the syntax is recognized by Prophet
mercado_prophet_df.columns = ["ds", "y"]    # Change the column labels to Prophet requirements (data series and y)

# Drop an NaN values from the prophet_df DataFrame
mercado_prophet_df = mercado_prophet_df.dropna()  # Drop NAs

# View the first and last five rows of the mercado_prophet_df DataFrame
display("Head",mercado_prophet_df.head())
display("Tail",mercado_prophet_df.tail())

In [ ]:
# Call the Prophet function, store as an object
model_mercado_trends = Prophet()   # Initialise Prophet model

In [ ]:
# Fit the time-series model.
model_mercado_trends.fit( mercado_prophet_df ) # Fit the df to the Prophet model

In [ ]:
# Create a future dataframe to hold predictions
# Make the prediction go out as far as 2000 hours (approx 80 days)
future_mercado_trends = model_mercado_trends.make_future_dataframe( periods=2000, freq="H" )   # Make future predictions dataframe 

# View the last five rows of the future_mercado_trends DataFrame
display("Tail",future_mercado_trends.tail())

In [ ]:
# Make the predictions for the trend data using the future_mercado_trends DataFrame
forecast_mercado_trends = model_mercado_trends.predict( future_mercado_trends ) # Use Prophet Predict on the dataframe

# Display the first five rows of the forecast_mercado_trends DataFrame
display("Head", forecast_mercado_trends.head())

#### Step 2: After estimating the model, plot the forecast. How's the near-term forecast for the popularity of MercadoLibre?

In [ ]:
# Plot the Prophet predictions for the Mercado trends data

fig8_label = "Figure 8 - Prophet predictions for Mercadolibre trends data"  # Set the plot's title

# Plotting using matplot lib was difficult to interpret. Have left the code in commented incase it may be of value.
# display(fig8_label)
# fig8 = model_mercado_trends.plot( forecast_mercado_trends ) # Plot model

# Implementation decicion: Preferred to use plot_plotly which allows zooming into desired areas of the plot for a more targetted view

# import plot_plotly libraries
from prophet.plot import plot_plotly, plot_components_plotly

# Generate the plot with a styled title
fig8 = plot_plotly( model_mercado_trends, forecast_mercado_trends, trend=True ).update_layout( title = dict( text = "<b>" + fig8_label + "</b>" ) ) # Generate the plot
# Code snippet to set the title adapted from: https://plotly.com/python/figure-labels/   then discovered that the fonts could be styled but not made bold, which is
# what I had intended for consistency with other plot titles. Further search revealed...
# Code snippet to include html styling adapted from user Maximillian Peters on https://stackoverflow.com/a/46516017

display(fig8) # Show the plot

##### Answer the following question:

**Question:**  How's the near-term forecast for the popularity of MercadoLibre?

**Answer:** The near-term forecast depicted in Figure 8 suggests based on the trend line that the searches are likely to decline.


#### Step 3: Plot the individual time series components of the model to answer the following questions:

* What time of day exhibits the greatest popularity?

* Which day of the week gets the most search traffic?

* What's the lowest point for search traffic in the calendar year?


In [ ]:
# Set the index in the forecast_mercado_trends DataFrame to the ds datetime column
forecast_mercado_trends = forecast_mercado_trends.set_index( "ds" )  # Set the index on the dataseries (date/time)

# View only the yhat,yhat_lower and yhat_upper columns from the DataFrame
display("Head", forecast_mercado_trends[["yhat", "yhat_lower", "yhat_upper"]].head())

Solutions Note: `yhat` represents the most likely (average) forecast, whereas `yhat_lower` and `yhat_upper` represents the worst and best case prediction (based on what are known as 95% confidence intervals).

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# From the forecast_mercado_trends DataFrame, use hvPlot to visualize
#  the yhat, yhat_lower, and yhat_upper columns over the last 2000 hours

fig9_label = "Figure 9 - Mercadolibre Forecasts - Average (yhat) Worst Case (yhat_lower) and Best Case (yhat_upper) predictions"  # Set the plot's title

# Create the Stock plot
fig9 = forecast_mercado_trends[["yhat", "yhat_lower", "yhat_upper"]].iloc[-2000:, :].hvplot(    # Plot the graph restricting to the last 2000 hours
    label=fig9_label,                                         # Set the title    
    xlabel="Date",                                            # Label the x axis
    ylabel="Forecast Trends",                                 # Label the y axis
    grid=True                                                 # Show the grid
)

display(fig9) # Show the plot

In [ ]:
# Reset the index in the forecast_mercado_trends DataFrame
forecast_mercado_trends = forecast_mercado_trends.reset_index() # Reset the index

# Use the plot_components function to visualize the forecast results
# for the ????? forecast_canada ????? ****(assuming this was meant to be forecast_mercado_trends)**** DataFrame
fig10_label = "Figure 10 - Mercadolibre Forecasts - Trend, Weekly, Yearly, and Daily"  # Set the plot's title

display( fig10_label ) # Show a title....need to further investigate setting the title. 
display( figures_10_mercado_trends = model_mercado_trends.plot_components(forecast_mercado_trends) ) # show the plot

##### Answer the following questions:

**Question:** What time of day exhibits the greatest popularity?

**Answer:** Midnight appears to be the most popular time for activity, based on Figure 10 Daily plot.

**Question:** Which day of week gets the most search traffic?
   
**Answer:** Tuesdays appear to be the day of week that get the most search traffic, based on Figure 10 Weekly plot.

**Question:** What's the lowest point for search traffic in the calendar year?

**Answer:** The lowest point for search traffic in the calendar year appears to be around mid October, based on Figure 10 Yearly plot. 


## Step 5 (Optional): Forecast Revenue by Using Time Series Models

A few weeks after your initial analysis, the finance group follows up to find out if you can help them solve a different problem. Your fame as a growth analyst in the company continues to grow!

Specifically, the finance group wants a forecast of the total sales for the next quarter. This will dramatically increase their ability to plan budgets and to help guide expectations for the company investors.

To do so, complete the following steps:

1. Read in the daily historical sales (that is, revenue) figures, and then apply a Prophet model to the data. The daily sales figures are quoted in millions of USD dollars.

2. Interpret the model output to identify any seasonal patterns in the company's revenue. For example, what are the peak revenue days? (Mondays? Fridays? Something else?)

3. Produce a sales forecast for the finance group. Give them a number for the expected total sales in the next quarter. Include the best- and worst-case scenarios to help them make better plans.




#### Step 1: Read in the daily historical sales (that is, revenue) figures, and then apply a Prophet model to the data.

In [ ]:
# Upload the "mercado_daily_revenue.csv" file into Colab, then store in a Pandas DataFrame
# Set the "date" column as the DatetimeIndex
# Sales are quoted in millions of US dollars

mercado_sales_filename = "mercado_daily_revenue.csv"    # set the filename to load
if using_colab:                                         # If in the Colab environment...
    from google.colab import files                      #     import colab library
    uploaded = files.upload()                           #     upload the file via colab

    df_mercado_sales =  pd.read_csv( mercado_sales_filename ) # Read the specified file into the dataframe
else:
    mercado_sales_path = Path("./Resources/" + mercado_sales_filename )    # Format the path as per OS's requirements
    df_mercado_sales = pd.read_csv(mercado_sales_path) # Read the specified file into the dataframe

df_mercado_sales = df_mercado_sales.set_index(pd.to_datetime(df_mercado_sales["date"], infer_datetime_format=True) ) # Set the index to the date
df_mercado_sales = df_mercado_sales.drop(columns=["date"]) # Drop the original Date column.... ("columns=labels" is equivalent to "labels, axis=1").

# Review the first and last five rows of the DataFrame
display("Head", df_mercado_sales.head()) # Show the first 5 rows
display("Tail", df_mercado_sales.tail()) # Show the last 5 rows

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Use hvPlot to visualize the daily sales figures
fig11_label = "Figure 11 - Mercadolibre Daily Sales Figures"  # Set the plot's title

# Create the Stock plot
fig11 = df_mercado_sales.hvplot(                      # Plot the graph
    label=fig11_label,                                # Set the title    
    xlabel="Date",                                    # Label the x axis
    ylabel="Daily Sales ($US Million)",               # Label the y axis
    grid=True,                                        # Show the grid
    xformatter=DatetimeTickFormatter(months="%b %Y"), # Show date in Month Year format
    yformatter=NumeralTickFormatter(format="0,0")     # Add thousands separator to the y axis
)

display(fig11) # Show the plot

In [ ]:
# Apply a Facebook Prophet model to the data.

# Set up the dataframe in the neccessary format:
# Reset the index so that date becomes a column in the DataFrame
mercado_sales_prophet_df = df_mercado_sales.reset_index()   # Reset the index

# Adjust the columns names to the Prophet syntax
mercado_sales_prophet_df.columns = ['ds', 'y'] # Set the colunms as needed by Prophet

# Visualize the DataFrame
display("Head", mercado_sales_prophet_df.head()) # Show the first 5 rows
display("Tail", mercado_sales_prophet_df.tail()) # Show the last 5 rows

In [ ]:
# Create the model
mercado_sales_prophet_model = Prophet()  # Create the Prophet model for the sales data

# Fit the model
mercado_sales_prophet_model.fit( mercado_sales_prophet_df ) # Fit the Prophet model for the sales data

In [ ]:
# Predict sales for 90 days (1 quarter) out into the future.

# Start by making a future dataframe
mercado_sales_prophet_future = mercado_sales_prophet_model.make_future_dataframe( periods=90, freq="D" )   # Make future predictions dataframe using 90 days as there appears to be no option for quarters 

# Display the last five rows of the future DataFrame
display("Tail", mercado_sales_prophet_future.tail())

In [ ]:
# Make predictions for the sales each day over the next quarter
mercado_sales_prophet_forecast = mercado_sales_prophet_model.predict( mercado_sales_prophet_future ) # Make the predictions

# Display the first 5 rows of the resulting DataFrame
display("Head", mercado_sales_prophet_forecast.head())

#### Step 2: Interpret the model output to identify any seasonal patterns in the company's revenue. For example, what are the peak revenue days? (Mondays? Fridays? Something else?)

In [ ]:
# Use the plot_components function to analyze seasonal patterns in the company's revenue

fig12_label = "Figure 12 - Prophet predictions for Mercadolibre sales data"  # Set the plot's title

display( fig12_label ) # Show a title....need to further investigate setting the title. 
display( figures_12_mercado_sales = mercado_sales_prophet_model.plot_components( mercado_sales_prophet_forecast) ) # show the plot

##### Answer the following question:

**Question:** For example, what are the peak revenue days? (Mondays? Fridays? Something else?)

**Answer:** Figure 12 Weekly view appears to suggest Mondays to Wednesdays are the most popular days of the week.

#### Step 3: Produce a sales forecast for the finance group. Give them a number for the expected total sales in the next quarter. Include the best- and worst-case scenarios to help them make better plans.

In [ ]:
# Plot the predictions for the Mercado sales
fig13_label = "Figure 13 - Prophet predictions for Mercadolibre sales data"  # Set the plot's title
fig13 = plot_plotly( mercado_sales_prophet_model, mercado_sales_prophet_forecast, trend=True ).update_layout( title = dict( text = "<b>" + fig13_label + "</b>" ) ) # Generate the plot
display(fig13) # Show the plot

In [ ]:
# For the mercado_sales_prophet_forecast DataFrame, set the ds column as the DataFrame Index
mercado_sales_prophet_forecast = mercado_sales_prophet_forecast.set_index( "ds" ) # Set the index to the ds (date/time) column

# Display the first and last five rows of the DataFrame
display("Head", mercado_sales_prophet_forecast.head()) # Show the first 5 rows
display("Tail", mercado_sales_prophet_forecast.tail()) # Show the last 5 rows

In [ ]:
# Produce a sales forecast for the finance division
# giving them a number for expected total sales next quarter.
# Provide best case (yhat_upper), worst case (yhat_lower), and most likely (yhat) scenarios.

# Create a forecast_quarter Dataframe for the period 2020-07-01 to 2020-09-30
# The DataFrame should include the columns yhat_upper, yhat_lower, and yhat
mercado_sales_forecast_quarter = mercado_sales_prophet_forecast.loc["2020-07-01":"2020-09-30"][["yhat_upper", "yhat_lower", "yhat" ]]   # Slice the df in the defined period, and use best, worst and most likely columns

# Update the column names for the forecast_quarter DataFrame
# to match what the finance division is looking for

sale_forecast_friendly_column_names = { "yhat_upper" : "Best Case", "yhat_lower" : "Worst Case", "yhat" : "Probable Case" }   # Create a dict to rename the columns

mercado_sales_forecast_quarter = mercado_sales_forecast_quarter.rename( columns = sale_forecast_friendly_column_names ) # Rename the columns to friendlier names

# Review the last five rows of the DataFrame
display("Tail", mercado_sales_forecast_quarter.tail()) # Show the last 5 rows

In [ ]:
# Display the summed values for all the rows in the forecast_quarter DataFrame

mercado_sales_forecast_quarter.sum() # present the title and the summary data

### Based on the forecast information generated above, produce a sales forecast for the finance division, giving them a number for expected total sales next quarter. Include best and worst case scenarios, to better help the finance team plan.

**Answer:** Based on Prophet forecasts using past sales data to predict future outcomes, the total sales predictions for the next quarter are:   
```
  Expected total sales:  (USD)   969.63M 
  Best case total sales: (USD) 1,052.14M 
  Worst case scenario:   (USD)   887.57M
```